# Coronavirus distribution maps in MX 
Source: Government open coronavirus database, INEGI coordenates catalog

In [2]:
###### ------------------- Main libraries -------------------################
import pandas as pd
import numpy as np 
from pprint import pprint
import folium 
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
import datetime

# Coordenates Catalog 

In [4]:
coordenadas = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/coordenadas/coordenadas.csv')
coordenadas.head()

C:\Users\Data Analyst\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Mapa,Cve_Ent,Nom_Ent,Nom_Abr,Cve_Mun,Nom_Mun,Cve_Loc,Nom_Loc,Ãmbito,Latitud,Longitud,Lat_Decimal,Lon_Decimal,Altitud,Cve_Carta,Pob_Total,Pob_Masculina,Pob_Femenina,Total De Viviendas Habitadas
0,10010001,1,Aguascalientes,Ags.,1,Aguascalientes,1,Aguascalientes,U,"21Â°52Â´47.362N""","102Â°17Â´45.768W""",21.879823,-102.296047,1878,F13D19,722250,348722,373528,185120
1,10010094,1,Aguascalientes,Ags.,1,Aguascalientes,94,Granja Adelita,R,"21Â°52Â´18.749N""","102Â°22Â´24.710W""",21.871875,-102.373531,1902,F13D18,14,*,*,2
2,10010096,1,Aguascalientes,Ags.,1,Aguascalientes,96,Agua Azul,R,"21Â°53Â´01.522N""","102Â°21Â´25.639W""",21.883756,-102.357122,1861,F13D18,37,21,16,11
3,10010100,1,Aguascalientes,Ags.,1,Aguascalientes,100,Rancho Alegre,R,"21Â°51Â´16.556N""","102Â°22Â´21.884W""",21.854599,-102.372746,1879,F13D18,10,*,*,1
4,10010102,1,Aguascalientes,Ags.,1,Aguascalientes,102,Los Arbolitos [Rancho],R,"21Â°46Â´48.650N""","102Â°21Â´26.261W""",21.780181,-102.357295,1861,F13D18,7,*,*,1


In [7]:
# Aguascalientes
coord_ags = coordenadas[coordenadas.Cve_Ent ==1]
coord_ags['MUNICIPIO'] = np.where(coord_ags.Nom_Mun == 'Aguascalientes','AGUASCALIENTES',
                                 np.where(coord_ags.Nom_Mun == 'Asientos','ASIENTOS',
                                 np.where(coord_ags.Nom_Mun == 'Calvillo','CALVILLO',
                                 np.where(coord_ags.Nom_Mun == 'CosÃ\xado','COSÍO',
                                 np.where(coord_ags.Nom_Mun == 'JesÃºs MarÃ\xada','JESÚS MARÍA',
                                 np.where(coord_ags.Nom_Mun == 'PabellÃ³n de Arteaga','PABELLÓN DE ARTEAGA',
                                 np.where(coord_ags.Nom_Mun == 'RincÃ³n de Romos','RINCÓN DE ROMOS',
                                 np.where(coord_ags.Nom_Mun == 'San JosÃ© de Gracia','SAN JOSÉ DE GRACIA',
                                 np.where(coord_ags.Nom_Mun == 'TepezalÃ¡','TEPEZALÁ',
                                 np.where(coord_ags.Nom_Mun == 'El Llano','EL LLANO',
                                 np.where(coord_ags.Nom_Mun == 'San Francisco de los Romo', 'SAN FRANCISCO DE LOS ROMO','Otro')))))))))))

C:\Users\Data Analyst\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [11]:
# Baja California
coord_bc = coordenadas[coordenadas.Cve_Ent ==2]
coord_bc['MUNICIPIO'] = np.where(coord_bc.Nom_Mun == 'Ensenada','ENSENADA',
                                 np.where(coord_bc.Nom_Mun == 'Mexicali','MEXICALI',
                                 np.where(coord_bc.Nom_Mun == 'Tecate','TECATE',
                                 np.where(coord_bc.Nom_Mun == 'Tijuana','TIJUANA',
                                 np.where(coord_bc.Nom_Mun == 'Playas de Rosarito','PLAYAS DE ROSARITO','NO ESPECIFICADO'
                                 )))))

C:\Users\Data Analyst\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [30]:
# Baja California Sur
coord_bcs = coordenadas[coordenadas.Cve_Ent == 3]
coord_bcs['MUNICIPIO'] = np.where(coord_bcs.Nom_Mun == 'La Paz','LA PAZ',
                                 np.where(coord_bcs.Nom_Mun == 'ComondÃº','COMONDÚ',
                                 np.where(coord_bcs.Nom_Mun == 'Los Cabos','LOS CABOS',
                                 np.where(coord_bcs.Nom_Mun == 'MulegÃ©','MULEGÉ',
                                 np.where(coord_bcs.Nom_Mun == 'Loreto','LORETO','NO ESPECIFICADO'
                                 )))))
coord_bcs.MUNICIPIO.value_counts()

C:\Users\Data Analyst\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


LA PAZ       1450
COMONDÚ       971
LOS CABOS     736
MULEGÉ        617
LORETO        208
Name: MUNICIPIO, dtype: int64

In [31]:
# Campeche
coord_camp = coordenadas[coordenadas.Cve_Ent == 4]
coord_camp['MUNICIPIO'] = np.where(coord_camp.Nom_Mun == 'Carmen','CARMEN',
                                 np.where(coord_camp.Nom_Mun == 'Candelaria','CANDELARIA',
                                 np.where(coord_camp.Nom_Mun == 'ChampotÃ³n','CHAMPOTÓN',
                                 np.where(coord_camp.Nom_Mun == 'EscÃ¡rcega','ESCÁRCEGA',
                                 np.where(coord_camp.Nom_Mun == 'Campeche','CAMPECHE',
                                 np.where(coord_camp.Nom_Mun == 'Calakmul','CALAKMUL',
                                 np.where(coord_camp.Nom_Mun == 'HopelchÃ©n','HOPELCHÉN',
                                 np.where(coord_camp.Nom_Mun == 'Palizada','PALIZADA',
                                 np.where(coord_camp.Nom_Mun == 'CalkinÃ','CALKINÍ',
                                 np.where(coord_bcs.Nom_Mun == 'HecelchakÃ¡n','HECELCHAKÁN',
                                 np.where(coord_bcs.Nom_Mun == 'Tenabo','TENABO','NO ESPECIFICADO'
                                 )))))))))))
coord_camp.MUNICIPIO.value_counts()

ValueError: Length of values does not match length of index

# MCP Catalog

In [5]:
municipios = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/coordenadas/catalogo_municipios.csv')
municipios.head()

,CLAVE_MUNICIPIO,MUNICIPIO,CLAVE_ENTIDAD
0,1,AGUASCALIENTES,1
1,2,ASIENTOS,1
2,3,CALVILLO,1
3,4,COSÍO,1
4,5,JESÚS MARÍA,1


In [6]:
# Aguascalientes
aguas_mcp = municipios[municipios.CLAVE_ENTIDAD ==1]
aguas_mcp.MUNICIPIO.values

array(['AGUASCALIENTES', 'ASIENTOS', 'CALVILLO', 'COSÍO', 'JESÚS MARÍA',
       'PABELLÓN DE ARTEAGA', 'RINCÓN DE ROMOS', 'SAN JOSÉ DE GRACIA',
       'TEPEZALÁ', 'EL LLANO', 'SAN FRANCISCO DE LOS ROMO',
       'NO ESPECIFICADO'], dtype=object)

In [12]:
# Baja California
bc_mcp = municipios[municipios.CLAVE_ENTIDAD ==2]
bc_mcp.MUNICIPIO.values

array(['ENSENADA', 'MEXICALI', 'TECATE', 'TIJUANA', 'PLAYAS DE ROSARITO',
       'NO ESPECIFICADO'], dtype=object)

In [13]:
# Baja California Sur
bcs_mcp = municipios[municipios.CLAVE_ENTIDAD ==3]
bcs_mcp.MUNICIPIO.values

array(['COMONDÚ', 'MULEGÉ', 'LA PAZ', 'LOS CABOS', 'LORETO',
       'NO ESPECIFICADO'], dtype=object)

In [14]:
# Campeche
camp_mcp = municipios[municipios.CLAVE_ENTIDAD ==4]
camp_mcp.MUNICIPIO.values

array(['CALKINÍ', 'CAMPECHE', 'CARMEN', 'CHAMPOTÓN', 'HECELCHAKÁN',
       'HOPELCHÉN', 'PALIZADA', 'TENABO', 'ESCÁRCEGA', 'CALAKMUL',
       'CANDELARIA', 'NO ESPECIFICADO'], dtype=object)

In [15]:
# Coahuila
coah_mcp = municipios[municipios.CLAVE_ENTIDAD ==5]
coah_mcp.MUNICIPIO.values

array(['ABASOLO', 'ACUÑA', 'ALLENDE', 'ARTEAGA', 'CANDELA', 'CASTAÑOS',
       'CUATRO CIÉNEGAS', 'ESCOBEDO', 'FRANCISCO I. MADERO', 'FRONTERA',
       'GENERAL CEPEDA', 'GUERRERO', 'HIDALGO', 'JIMÉNEZ', 'JUÁREZ',
       'LAMADRID', 'MATAMOROS', 'MONCLOVA', 'MORELOS', 'MÚZQUIZ',
       'NADADORES', 'NAVA', 'OCAMPO', 'PARRAS', 'PIEDRAS NEGRAS',
       'PROGRESO', 'RAMOS ARIZPE', 'SABINAS', 'SACRAMENTO', 'SALTILLO',
       'SAN BUENAVENTURA', 'SAN JUAN DE SABINAS', 'SAN PEDRO',
       'SIERRA MOJADA', 'TORREÓN', 'VIESCA', 'VILLA UNIÓN', 'ZARAGOZA',
       'NO ESPECIFICADO'], dtype=object)

In [16]:
# Colima
col_mcp = municipios[municipios.CLAVE_ENTIDAD ==6]
col_mcp.MUNICIPIO.values

array(['ARMERÍA', 'COLIMA', 'COMALA', 'COQUIMATLÁN', 'CUAUHTÉMOC',
       'IXTLAHUACÁN', 'MANZANILLO', 'MINATITLÁN', 'TECOMÁN',
       'VILLA DE ÁLVAREZ', 'NO ESPECIFICADO'], dtype=object)

In [17]:
# Chiapas
chiap_mcp = municipios[municipios.CLAVE_ENTIDAD ==7]
chiap_mcp.MUNICIPIO.values

array(['ACACOYAGUA', 'ACALA', 'ACAPETAHUA', 'ALTAMIRANO', 'AMATÁN',
       'AMATENANGO DE LA FRONTERA', 'AMATENANGO DEL VALLE',
       'ANGEL ALBINO CORZO', 'ARRIAGA', 'BEJUCAL DE OCAMPO',
       'BELLA VISTA', 'BERRIOZÁBAL', 'BOCHIL', 'EL BOSQUE', 'CACAHOATÁN',
       'CATAZAJÁ', 'CINTALAPA', 'COAPILLA', 'COMITÁN DE DOMÍNGUEZ',
       'LA CONCORDIA', 'COPAINALÁ', 'CHALCHIHUITÁN', 'CHAMULA', 'CHANAL',
       'CHAPULTENANGO', 'CHENALHÓ', 'CHIAPA DE CORZO', 'CHIAPILLA',
       'CHICOASÉN', 'CHICOMUSELO', 'CHILÓN', 'ESCUINTLA',
       'FRANCISCO LEÓN', 'FRONTERA COMALAPA', 'FRONTERA HIDALGO',
       'LA GRANDEZA', 'HUEHUETÁN', 'HUIXTÁN', 'HUITIUPÁN', 'HUIXTLA',
       'LA INDEPENDENCIA', 'IXHUATÁN', 'IXTACOMITÁN', 'IXTAPA',
       'IXTAPANGAJOYA', 'JIQUIPILAS', 'JITOTOL', 'JUÁREZ', 'LARRÁINZAR',
       'LA LIBERTAD', 'MAPASTEPEC', 'LAS MARGARITAS', 'MAZAPA DE MADERO',
       'MAZATÁN', 'METAPA', 'MITONTIC', 'MOTOZINTLA', 'NICOLÁS RUÍZ',
       'OCOSINGO', 'OCOTEPEC', 'OCOZOCOAUTLA DE ESPI

In [18]:
# Chihuahua
chih_mcp = municipios[municipios.CLAVE_ENTIDAD ==8]
chih_mcp.MUNICIPIO.values

array(['AHUMADA', 'ALDAMA', 'ALLENDE', 'AQUILES SERDÁN', 'ASCENSIÓN',
       'BACHÍNIVA', 'BALLEZA', 'BATOPILAS DE MANUEL GÓMEZ MORÍN',
       'BOCOYNA', 'BUENAVENTURA', 'CAMARGO', 'CARICHÍ', 'CASAS GRANDES',
       'CORONADO', 'COYAME DEL SOTOL', 'LA CRUZ', 'CUAUHTÉMOC',
       'CUSIHUIRIACHI', 'CHIHUAHUA', 'CHÍNIPAS', 'DELICIAS',
       'DR. BELISARIO DOMÍNGUEZ', 'GALEANA', 'SANTA ISABEL',
       'GÓMEZ FARÍAS', 'GRAN MORELOS', 'GUACHOCHI', 'GUADALUPE',
       'GUADALUPE Y CALVO', 'GUAZAPARES', 'GUERRERO',
       'HIDALGO DEL PARRAL', 'HUEJOTITÁN', 'IGNACIO ZARAGOZA', 'JANOS',
       'JIMÉNEZ', 'JUÁREZ', 'JULIMES', 'LÓPEZ', 'MADERA', 'MAGUARICHI',
       'MANUEL BENAVIDES', 'MATACHÍ', 'MATAMOROS', 'MEOQUI', 'MORELOS',
       'MORIS', 'NAMIQUIPA', 'NONOAVA', 'NUEVO CASAS GRANDES', 'OCAMPO',
       'OJINAGA', 'PRAXEDIS G. GUERRERO', 'RIVA PALACIO', 'ROSALES',
       'ROSARIO', 'SAN FRANCISCO DE BORJA', 'SAN FRANCISCO DE CONCHOS',
       'SAN FRANCISCO DEL ORO', 'SANTA BÁRBARA', 'SATEVÓ

In [19]:
# CDMX
cdmx_mcp = municipios[municipios.CLAVE_ENTIDAD ==9]
cdmx_mcp.MUNICIPIO.values

array(['AZCAPOTZALCO', 'COYOACÁN', 'CUAJIMALPA DE MORELOS',
       'GUSTAVO A. MADERO', 'IZTACALCO', 'IZTAPALAPA',
       'LA MAGDALENA CONTRERAS', 'MILPA ALTA', 'ÁLVARO OBREGÓN',
       'TLÁHUAC', 'TLALPAN', 'XOCHIMILCO', 'BENITO JUÁREZ', 'CUAUHTÉMOC',
       'MIGUEL HIDALGO', 'VENUSTIANO CARRANZA', 'NO ESPECIFICADO'],
      dtype=object)

In [20]:
# Durango
dur_mcp = municipios[municipios.CLAVE_ENTIDAD ==10]
dur_mcp.MUNICIPIO.values

array(['CANATLÁN', 'CANELAS', 'CONETO DE COMONFORT', 'CUENCAMÉ',
       'DURANGO', 'GENERAL SIMÓN BOLÍVAR', 'GÓMEZ PALACIO',
       'GUADALUPE VICTORIA', 'GUANACEVÍ', 'HIDALGO', 'INDÉ', 'LERDO',
       'MAPIMÍ', 'MEZQUITAL', 'NAZAS', 'NOMBRE DE DIOS', 'OCAMPO',
       'EL ORO', 'OTÁEZ', 'PÁNUCO DE CORONADO', 'PEÑÓN BLANCO', 'POANAS',
       'PUEBLO NUEVO', 'RODEO', 'SAN BERNARDO', 'SAN DIMAS',
       'SAN JUAN DE GUADALUPE', 'SAN JUAN DEL RÍO',
       'SAN LUIS DEL CORDERO', 'SAN PEDRO DEL GALLO', 'SANTA CLARA',
       'SANTIAGO PAPASQUIARO', 'SÚCHIL', 'TAMAZULA', 'TEPEHUANES',
       'TLAHUALILO', 'TOPIA', 'VICENTE GUERRERO', 'NUEVO IDEAL',
       'NO ESPECIFICADO'], dtype=object)

# MX Covid Data

In [10]:
covid = pd.read_csv('C:/Users/Data Analyst/Documents/Otros/COVID-19_MX/covid_mx_12_05_20_ac1.csv')
covid.head(2)

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OBESIDAD,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,DEFUNCION
0,12/05/2020,0ae84c,2,1,9,1,9,9,16,1,...,2,2,2,99,1,99,MÃ©xico,99,97,0
1,12/05/2020,1.14E+15,2,3,2,2,2,2,2,1,...,2,2,2,1,1,99,MÃ©xico,99,97,1
